In [1]:
import numpy as np

In [2]:
from model_keras import S2sModel

Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [3]:
from seq_seq_annot_DS1DS2 import read_mitbih, numerize_y, oversampling


In [4]:
model = S2sModel(num_encoder_tokens=280, num_decoder_tokens=4)

In [5]:
filename = 'data/s2s_mitbih_aami_DS1DS2'
n_oversampling = 6000
max_time = 10
classes = ['N', 'S', 'V']

In [6]:
X_train, y_train = read_mitbih(filename, max_time, classes=classes, max_nlabel=50000, trainset=1)
X_test, y_test = read_mitbih(filename, max_time, classes=classes, max_nlabel=50000, trainset=0)

# input_dim
input_depth = X_train.shape[2]
n_channels = 10
print("# of sequences: ", len(X_train))

classes = np.unique(y_train)
n_classes = len(classes)

char2numY = dict(zip(classes, range(len(classes))))
char2numY['<GO>'] = len(char2numY)
num2charY = dict(zip(char2numY.values(), char2numY.keys()))
# char2numY = {'N': 0, 'S': 1, 'V': 2, '<GO>': 3}
# num2charY = {0: 'N', 1: 'S', 2: 'V', 3: '<GO>'}

y_train, y_test = numerize_y(y_train, y_test, char2numY)

x_seq_length = len(X_train[0])
y_seq_length = len(y_train[0]) - 1  # why minus one ?

if n_oversampling:
    print("oversampling ...")
    X_train, y_train = oversampling(X_train, y_train, n_oversampling, char2numY)



Records processed!
Records processed!
# of sequences:  5052
oversampling ...


In [7]:
print(X_train.shape)
print(y_train.shape)

(5879, 10, 280)
(5879, 11)


In [8]:
def num2onehot(y, l):
    y_one = np.zeros(l)
    y_one[y] = 1
    return y_one
    

In [9]:
y_train[0]

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
encoder_input = X_train.reshape(5879, 10, 28, 10)
num_cls = max(y_train.ravel())+1
decoder_input = np.array([[num2onehot(yy, num_cls) for yy in y] for y in y_train])
y_train_target = decoder_input[:, 1:, :]
target_end = decoder_input[:, -2:-1, :]
decoder_target = np.concatenate((y_train_target, target_end), axis=1)
print(decoder_input.shape)
print(decoder_target.shape)

(5879, 11, 4)
(5879, 11, 4)


In [ ]:
model.fit(encoder_input, decoder_input, decoder_target)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 5291 samples, validate on 588 samples
Epoch 1/12
5291/5291 [==============================] - 9s 2ms/step - loss: 0.2248 - accuracy: 0.9204 - val_loss: 0.1594 - val_accuracy: 0.9501
Epoch 2/12
5291/5291 [==============================] - 7s 1ms/step - loss: 0.1533 - accuracy: 0.9430 - val_loss: 0.1545 - val_accuracy: 0.9465
Epoch 3/12
5291/5291 [==============================] - 7s 1ms/step - loss: 0.1207 - accuracy: 0.9562 - val_loss: 0.1036 - val_accuracy: 0.9796
Epoch 4/12
5291/5291 [==============================] - 7s 1ms/step - loss: 0.0994 - accuracy: 0.9640 - val_loss: 0.0269 - val_accuracy: 0.9975
Epoch 5/12
5291/5291 [==============================] - 7s 1ms/step - loss: 0.0839 - accuracy: 0.9694 - val_loss: 0.0134 - val_accuracy: 0.9994
Epoch 6/12
5291/5291 [==============================] - 7s 1ms/step - loss: 0.0712 - accuracy: 0.9749 - val_loss: 0.0433 - val_accuracy: 0

In [ ]:
y_pred = model.predict(encoder_input[0:1])

In [ ]:
[np.argmax(y) for y in y_pred]

In [ ]:
[np.argmax(y) for y in  y_train_target[0]]